In [2]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [3]:
# Path to the folder containing the CSV files
path = "data/data_ilr_transformed/"

# Generate a list of file paths for the long datasets
file_list = [f"{path}d_study{i}_long.csv" for i in range(1, 9)]

# Read and combine all datasets into one DataFrame
df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

# Check the resulting combined DataFrame
print(f"Combined dataset shape: {df.shape}")

# Variable lists
ilr_columns = ['ilr1', 'ilr2', 'ilr3']
control_columns = ['excluded', 'injustice', 'personal', 'violence']

# Eliminating NAs
df = df[df['gender'].notna()]
df = df[df['age'].notna()]

Combined dataset shape: (15424, 18)


In [5]:
df['time'].unique()

array([ 1,  2,  3,  5,  6,  7,  9, 10, 11, 13, 14, 15])

In [ ]:
# Creating -1 and -2 lag features for each ilr variable
for col in ilr_columns:
    df[f'{col}_lag1'] = df.groupby('ID')[col].shift(1)
    df[f'{col}_lag2'] = df.groupby('ID')[col].shift(2)

# Calculate mean and standard deviation per participant (ID) for each ilr column
for col in ilr_columns:
    # Mean per participant
    df[f'{col}_mean'] = df.groupby('ID')[col].transform('mean')
    
    # Standard deviation per participant
    df[f'{col}_std'] = df.groupby('ID')[col].transform('std')
    
    # Minimum and maximum values per participant
    df[f'{col}_min'] = df.groupby('ID')[col].transform('min')
    df[f'{col}_max'] = df.groupby('ID')[col].transform('max')


# Additional time-based feature: time point relative to the total duration for each participant
# This provides a scaled measure of progression within the time series
df['relative_time'] = df.groupby('ID')['time'].transform(lambda x: x / x.max())


# Optional cumulative feature: cumulative sum of the 'ilr' values within each participant's time series
# This can capture a trend or overall trajectory for each ilr over time
#for col in ilr_columns:
#    df[f'{col}_cumsum'] = df.groupby('ID')[col].cumsum()


## Response Variability Over Time:
#for col in ilr_columns:
#    df[f'{col}_delta'] = df.groupby('ID')[col].diff().fillna(0)  # Absolute change from previous time point


## Interaction Terms: Control Variables with Time
#for col in control_columns:
#    df[f'{col}_time_interaction'] = df[col] * df['time']


# Calculate deviation from the mean of each ilr value within each experiment and time point
#for col in ilr_columns:
#    df[f'{col}_deviation_from_group'] = df[col] - df.groupby(['Experiment', 'time'])[col].transform('mean')


# Calculate the trend/slope for each ilr column
#from sklearn.linear_model import LinearRegression
#for col in ilr_columns:
#    df[f'{col}_trend'] = df.groupby('ID').apply(lambda x: 
#        LinearRegression().fit(x[['time']], x[col]).coef_[0] if len(x) > 1 else np.nan).reset_index(level=0, drop=True)


# Moving Average or Exponential Moving Average (EMA)
window_size = 3  # Define your preferred window size
for col in ilr_columns:
    df[f'{col}_moving_avg'] = df.groupby('ID')[col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    # df[f'{col}_ema'] = df.groupby('ID')[col].transform(lambda x: x.ewm(span=window_size, adjust=False).mean())



from statsmodels.regression.mixed_linear_model import MixedLM

# List of ilr columns
ilr_columns = ['ilr1', 'ilr2', 'ilr3']
ilr_moving_avg_cols = ['ilr1_moving_avg', 'ilr2_moving_avg', 'ilr3_moving_avg']

# Residualize each ilr variable
for col in ilr_moving_avg_cols:
    # Fit the mixed-effects model
    model = MixedLM.from_formula(f"{col} ~ relative_time", groups="Experiment", data=df)
    result = model.fit()
    
    # Add residuals to the DataFrame
    df[f'{col}_residual'] = df[col] - result.fittedvalues

ilr_resid_columns = ['ilr1_residual', 'ilr2_residual', 'ilr3_residual']






import numpy as np
import pandas as pd

# Number of Fourier components to retain
fourier_components = 1

ilr_moving_avg_residual_cols = ['ilr1_moving_avg_residual', 'ilr2_moving_avg_residual', 'ilr3_moving_avg_residual']

# Compute Fourier transform for each 'ilr' variable per participant
for col in ilr_moving_avg_residual_cols:
    def compute_fourier(x):
        fft_vals = np.fft.fft(x)
        # Retain only the first N components (real and imaginary parts)
        return np.hstack([fft_vals.real[:fourier_components], fft_vals.imag[:fourier_components]])

    # Apply Fourier transformation and store as separate columns
    fourier_df = (
        df.groupby('ID')[col]
        .apply(lambda x: compute_fourier(x.values))
        .apply(pd.Series)
        .rename(columns=lambda i: f'{col}_fourier_{i+1}')
    )

    # Add Fourier features back to the dataframe
    df = df.join(fourier_df, on='ID')


In [ ]:
# Step 2: Prepare Time Series Data for Clustering
# Collect each participant’s data with the new features into a standardized time series format

time_series_data = []
grouped = df.groupby("ID")

# Define the feature columns to include in time series
feature_columns = [
    # 'condition', 
    'gender', 'age',
    #'ilr1', 'ilr2', 'ilr3', 
    'excluded', 'injustice', 'personal', 'violence',
    #'ilr1_lag1', 'ilr2_lag1', 'ilr3_lag1', 'ilr1_lag2', 'ilr2_lag2', 'ilr3_lag2',
    #'ilr1_mean', 'ilr2_mean', 'ilr3_mean', 'ilr1_std', 'ilr2_std', 'ilr3_std',
    #'ilr1_min', 'ilr2_min', 'ilr3_min', 'ilr1_max', 'ilr2_max', 'ilr3_max',
    'relative_time', 
    ######'ilr1_cumsum', 'ilr2_cumsum', 'ilr3_cumsum', 
    ######'ilr1_delta', 'ilr2_delta', 'ilr3_delta',  ### definitely nono
    #'excluded_time_interaction', 'injustice_time_interaction', 'personal_time_interaction', 'violence_time_interaction',
    #'ilr1_deviation_from_group', 'ilr2_deviation_from_group', 'ilr3_deviation_from_group',
    #'ilr1_trend', 'ilr2_trend', 'ilr3_trend', 
    # 'ilr1_moving_avg', 'ilr2_moving_avg', 'ilr3_moving_avg', 
    ######'ilr1_ema', 'ilr2_ema', 'ilr3_ema',
    #'ilr1_residual', 'ilr2_residual', 'ilr3_residual',
    #'ilr1_fourier_1', 'ilr2_fourier_1', 'ilr3_fourier_1',
    #'ilr1_fourier_2', 'ilr2_fourier_2', 'ilr3_fourier_2'
    # 'ilr1_moving_avg_fourier_1', 'ilr2_moving_avg_fourier_2', 'ilr3_moving_avg_fourier_1',
    'ilr1_moving_avg_residual_fourier_1',
    #'ilr1_moving_avg_residual_fourier_2',
    'ilr2_moving_avg_residual_fourier_1',
    #'ilr2_moving_avg_residual_fourier_2',
    'ilr3_moving_avg_residual_fourier_1',
    #'ilr3_moving_avg_residual_fourier_2'

]

# Preprocess the time series data for each participant
for participant, group in grouped:
    group_sorted = group.sort_values("time")
    time_series = group_sorted[feature_columns].values
    
    # Pad or truncate each participant's time series to the same length
    max_length = 16  # Adjust as necessary
    if len(time_series) < max_length:
        time_series = np.pad(time_series, ((0, max_length - len(time_series)), (0, 0)), 'constant', constant_values=np.nan)
    else:
        time_series = time_series[:max_length]
    
    time_series_data.append(time_series)

# Convert to numpy array for clustering
time_series_data = np.array(time_series_data)